# Building Our First Envelope

In [1]:
#add ../src to the path ready for python to find it
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

## Set up an Envelope using constructors

In [2]:
from openfloor.envelope import *
from openfloor.events import *
from openfloor.dialog_event import *

#create an event
dialog_event = DialogEvent.from_json('{"speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2024-03-14T12:00:00.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "i want to go to vancouver"}]}}}')
utterance_event=UtteranceEvent(to=To(speakerUri="tag:userproxy.com,2025:abc123", serviceUrl="https://userproxy.com"),dialogEvent=dialog_event)
print(utterance_event.to_json(indent=2))

{
  "eventType": "utterance",
  "to": {
    "speakerUri": "tag:userproxy.com,2025:abc123",
    "serviceUrl": "https://userproxy.com"
  },
  "parameters": {
    "dialogEvent": {
      "id": "de:bd541a12-31ec-4e95-abfa-dd024e4d753c",
      "speakerUri": "tag:userproxy.com,2025:abc123",
      "span": {
        "startTime": "2025-05-10T00:05:06.517133"
      },
      "features": {
        "text": {
          "mimeType": "text/plain",
          "tokens": [
            {
              "value": "i want to go to vancouver"
            }
          ]
        }
      }
    }
  }
}


# Building Envelope

## Define the manifests

The openfloor standard uses a predefined format to carry information about conversants in a conversation.   These are called Manifests.

The Manifest is split into two parts, an `identification` section and a `capabilities` section.  The `identification` section is the part that is actually used in the envelope to carry information about conversants in the conversation but we will define a full manifest for our agent to show all the parts.

There is no difference between users and agents in the openfloor standard so we will define our user and our agent.  Remember that the standard can support multiple simulataneous users and agents conversing around the floor.

In [3]:
from openfloor.manifest import *

chat_agent_details = Manifest(
  identification = Identification(
    speakerUri="tag:dev.travelbot,2025:0001", 
    serviceUrl="https://dev.travelbot.ee/openfloor/conversation",
    organization="Travelbot Inc.",
    conversationalName="travelbot",
    department="Reservations and Customer Service",
    role="Reservation Specialist",
    synopsis="Reservation specialist as part of the Travelbot system."),
  capabilities = [
    Capability(
        keyphrases=["book","reserve","flight","hotel","car","cruise"], 
        descriptions=["book flights, hotels, cars, and cruises"], 
        languages=["en-us"],
        supportedLayers=SupportedLayers(input=["text"], output=["text"])
    ),
    Capability(
        keyphrases=["billing","payment","invoice","receipt"], 
        descriptions=["Respond to enquiries about billing, payments, and invoices"], 
        languages=["en-us"],
        supportedLayers=SupportedLayers(input=["text"], output=["text"])
    )
  ]
)

user_details = Manifest(
  Identification(
            conversationalName="John Doe",
            speakerUri="tag:userproxy.com,2025:abc123", 
            serviceUrl="https://userproxy.com",
            role="User"
        )
)

#print(chat_agent_details.to_json(indent=2))
#print(user_details.to_json(indent=2))

print(chat_agent_details)
print(user_details)

Manifest(identification=Identification(speakerUri='tag:dev.travelbot,2025:0001', serviceUrl='https://dev.travelbot.ee/openfloor/conversation', organization='Travelbot Inc.', conversationalName='travelbot', department='Reservations and Customer Service', role='Reservation Specialist', synopsis='Reservation specialist as part of the Travelbot system.'), capabilities=[Capability(keyphrases=['book', 'reserve', 'flight', 'hotel', 'car', 'cruise'], descriptions=['book flights, hotels, cars, and cruises'], languages=['en-us'], supportedLayers=SupportedLayers(input=['text'], output=['text'])), Capability(keyphrases=['billing', 'payment', 'invoice', 'receipt'], descriptions=['Respond to enquiries about billing, payments, and invoices'], languages=['en-us'], supportedLayers=SupportedLayers(input=['text'], output=['text']))])
Manifest(identification=Identification(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl='https://userproxy.com', organization=None, conversationalName='John Doe', depa

## Create an Empty Envelope

Here we define the minimal empty conversation envelope.  All envelopes must be assigned to a conversation and have a sender.

In [4]:
from openfloor import UtteranceEvent

conversation=Conversation()
sender=Sender(speakerUri=user_details.identification.speakerUri)
envelope=Envelope(conversation=conversation,sender=sender)

print(envelope.to_json(indent=2))

{
  "schema": {
    "version": "1.0.0"
  },
  "conversation": {
    "id": "conv:166aea61-7d20-4414-8d17-f62d048000a8"
  },
  "sender": {
    "speakerUri": "tag:userproxy.com,2025:abc123"
  },
  "events": []
}


## Add Conversants

In [5]:
chat_agent_persistent_state=PersistentState(
  conversationEnded=None,
  conversationActive=True,
  conversationPaused=False,
  conversationResumed=None
)
conversation.conversants.append(Conversant(user_details.identification))
conversation.conversants.append(Conversant(chat_agent_details.identification,persistentState=chat_agent_persistent_state))
#print(envelope.to_json(indent=2))
print(envelope)
print(dict(envelope))

Envelope(conversation=Conversation(id='conv:166aea61-7d20-4414-8d17-f62d048000a8', conversants=[Conversant(identification=Identification(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl='https://userproxy.com', organization=None, conversationalName='John Doe', department=None, role='User', synopsis=None), persistentState={}), Conversant(identification=Identification(speakerUri='tag:dev.travelbot,2025:0001', serviceUrl='https://dev.travelbot.ee/openfloor/conversation', organization='Travelbot Inc.', conversationalName='travelbot', department='Reservations and Customer Service', role='Reservation Specialist', synopsis='Reservation specialist as part of the Travelbot system.'), persistentState={})]), sender=Sender(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl=None), schema=Schema(version='1.0.0', url=None), events=[])
{'schema': {'version': '1.0.0'}, 'conversation': {'id': 'conv:166aea61-7d20-4414-8d17-f62d048000a8', 'conversants': [{'identification': {'speakerUri': 'tag:us

### Add UtteranceEvent

In [6]:
from openfloor.envelope import *

utterance=DialogEvent(speakerUri=user_details.identification.speakerUri,features={"text":TextFeature(values=["Give me the times to Vancouver!"])})
envelope.events.append(UtteranceEvent(dialogEvent=utterance))

#print(envelope.to_json(indent=2))
print(envelope)
print(dict(envelope))

Envelope(conversation=Conversation(id='conv:166aea61-7d20-4414-8d17-f62d048000a8', conversants=[Conversant(identification=Identification(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl='https://userproxy.com', organization=None, conversationalName='John Doe', department=None, role='User', synopsis=None), persistentState={}), Conversant(identification=Identification(speakerUri='tag:dev.travelbot,2025:0001', serviceUrl='https://dev.travelbot.ee/openfloor/conversation', organization='Travelbot Inc.', conversationalName='travelbot', department='Reservations and Customer Service', role='Reservation Specialist', synopsis='Reservation specialist as part of the Travelbot system.'), persistentState={})]), sender=Sender(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl=None), schema=Schema(version='1.0.0', url=None), events=[UtteranceEvent(eventType='utterance', to=None, reason=None, parameters=<openfloor.envelope.Parameters object at 0x7fbe9460e210>)])
{'schema': {'version': '1.0.0'

In [7]:
utterance=DialogEvent(
  speakerUri=user_details.identification.speakerUri,
  features={"text":TextFeature(values=["Give me the times to Vancouver!"])}
  )
envelope.events[0]=(
  UtteranceEvent(
    dialogEvent=utterance,
    to=To(
      speakerUri=chat_agent_details.identification.speakerUri,
      private=True
    )
  )
)

#print(envelope.to_json(indent=2,as_payload=True))
print(envelope)
print(dict(envelope))

Envelope(conversation=Conversation(id='conv:166aea61-7d20-4414-8d17-f62d048000a8', conversants=[Conversant(identification=Identification(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl='https://userproxy.com', organization=None, conversationalName='John Doe', department=None, role='User', synopsis=None), persistentState={}), Conversant(identification=Identification(speakerUri='tag:dev.travelbot,2025:0001', serviceUrl='https://dev.travelbot.ee/openfloor/conversation', organization='Travelbot Inc.', conversationalName='travelbot', department='Reservations and Customer Service', role='Reservation Specialist', synopsis='Reservation specialist as part of the Travelbot system.'), persistentState={})]), sender=Sender(speakerUri='tag:userproxy.com,2025:abc123', serviceUrl=None), schema=Schema(version='1.0.0', url=None), events=[UtteranceEvent(eventType='utterance', to=To(speakerUri='tag:dev.travelbot,2025:0001', serviceUrl=None, private=True), reason=None, parameters=<openfloor.envelope

## Add a ContextEvent containing DialogHistory

In [8]:
from openfloor.envelope import *
from openfloor.events import *
from openfloor.dialog_event import *

dialog_history=DialogHistory()
dialog_history.append(DialogEvent.from_json('{"id": "event-1", "speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2024-03-14T12:00:00.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "hello"}]}}}'))
dialog_history.append(DialogEvent.from_json('{"id": "event-2", "speakerUri": "tag:dev.travelbot,2025:0001", "span": {"startTime": "2024-03-14T12:04:00.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "hello, how can i help you?"}]}}}'))
dialog_history.append(DialogEvent.from_json('{"id": "event-3", "speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2024-03-14T12:00:05.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "i need to book a flight"}]}}}'))
dialog_history.append(DialogEvent.from_json('{"id": "event-4", "speakerUri": "tag:dev.travelbot,2025:0001", "span": {"startTime": "2024-03-14T12:12:00.000000"}, "features": {"text": {"mimeType": "text/plain", "tokens": [{"value": "i can help you with that"}]}}}'))

context_event=ContextEvent(dialogHistory=dialog_history)
context_event.parameters["arbitrary_key"]="arbitrary_value"
#envelope.events.append(context_event)

#print the envelope
print(envelope.to_json(indent=2,as_payload=True))

{
  "openFloor": {
    "schema": {
      "version": "1.0.0"
    },
    "conversation": {
      "id": "conv:166aea61-7d20-4414-8d17-f62d048000a8",
      "conversants": [
        {
          "identification": {
            "speakerUri": "tag:userproxy.com,2025:abc123",
            "serviceUrl": "https://userproxy.com",
            "conversationalName": "John Doe",
            "role": "User"
          }
        },
        {
          "identification": {
            "speakerUri": "tag:dev.travelbot,2025:0001",
            "serviceUrl": "https://dev.travelbot.ee/openfloor/conversation",
            "organization": "Travelbot Inc.",
            "conversationalName": "travelbot",
            "department": "Reservations and Customer Service",
            "role": "Reservation Specialist",
            "synopsis": "Reservation specialist as part of the Travelbot system."
          }
        }
      ]
    },
    "sender": {
      "speakerUri": "tag:userproxy.com,2025:abc123"
    },
    "events": [

##  To/From 

In [9]:
#Convert to dict (i.e. python object containing simple JSON equivalent types)
dict1=dict(envelope)
print(dict1)

envelope_from_dict=Envelope.from_dict(dict1)
print(dict(envelope_from_dict))

{'schema': {'version': '1.0.0'}, 'conversation': {'id': 'conv:166aea61-7d20-4414-8d17-f62d048000a8', 'conversants': [{'identification': {'speakerUri': 'tag:userproxy.com,2025:abc123', 'serviceUrl': 'https://userproxy.com', 'conversationalName': 'John Doe', 'role': 'User'}}, {'identification': {'speakerUri': 'tag:dev.travelbot,2025:0001', 'serviceUrl': 'https://dev.travelbot.ee/openfloor/conversation', 'organization': 'Travelbot Inc.', 'conversationalName': 'travelbot', 'department': 'Reservations and Customer Service', 'role': 'Reservation Specialist', 'synopsis': 'Reservation specialist as part of the Travelbot system.'}}]}, 'sender': {'speakerUri': 'tag:userproxy.com,2025:abc123'}, 'events': [{'eventType': 'utterance', 'to': {'speakerUri': 'tag:dev.travelbot,2025:0001', 'private': True}, 'parameters': {'dialogEvent': {'id': 'de:885bec8f-03d7-49b2-bed0-75bbf1d257c2', 'speakerUri': 'tag:userproxy.com,2025:abc123', 'span': {'startTime': '2025-05-10T00:05:06.575947'}, 'features': {'text'

In [10]:
#Convert to JSON string
json1=envelope.to_json(as_payload=True)
print(json1)

#Create from JSON string
envelope_from_json=Envelope.from_json(json1,as_payload=True)
print(envelope_from_json.to_json(as_payload=True))

{"openFloor": {"schema": {"version": "1.0.0"}, "conversation": {"id": "conv:166aea61-7d20-4414-8d17-f62d048000a8", "conversants": [{"identification": {"speakerUri": "tag:userproxy.com,2025:abc123", "serviceUrl": "https://userproxy.com", "conversationalName": "John Doe", "role": "User"}}, {"identification": {"speakerUri": "tag:dev.travelbot,2025:0001", "serviceUrl": "https://dev.travelbot.ee/openfloor/conversation", "organization": "Travelbot Inc.", "conversationalName": "travelbot", "department": "Reservations and Customer Service", "role": "Reservation Specialist", "synopsis": "Reservation specialist as part of the Travelbot system."}}]}, "sender": {"speakerUri": "tag:userproxy.com,2025:abc123"}, "events": [{"eventType": "utterance", "to": {"speakerUri": "tag:dev.travelbot,2025:0001", "private": true}, "parameters": {"dialogEvent": {"id": "de:885bec8f-03d7-49b2-bed0-75bbf1d257c2", "speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2025-05-10T00:05:06.575947"}, "feat

In [13]:
#Save as JSON
envelope.to_file("../sample_json/envelope1.json",as_payload=True)
print(envelope.to_json(as_payload=True))

#Create from dictionary
envelope_from_file=Envelope.from_file("../sample_json/envelope1.json",as_payload=True)
print(envelope_from_file.to_json(as_payload=True))

{"openFloor": {"schema": {"version": "1.0.0"}, "conversation": {"id": "conv:166aea61-7d20-4414-8d17-f62d048000a8", "conversants": [{"identification": {"speakerUri": "tag:userproxy.com,2025:abc123", "serviceUrl": "https://userproxy.com", "conversationalName": "John Doe", "role": "User"}}, {"identification": {"speakerUri": "tag:dev.travelbot,2025:0001", "serviceUrl": "https://dev.travelbot.ee/openfloor/conversation", "organization": "Travelbot Inc.", "conversationalName": "travelbot", "department": "Reservations and Customer Service", "role": "Reservation Specialist", "synopsis": "Reservation specialist as part of the Travelbot system."}}]}, "sender": {"speakerUri": "tag:userproxy.com,2025:abc123"}, "events": [{"eventType": "utterance", "to": {"speakerUri": "tag:dev.travelbot,2025:0001", "private": true}, "parameters": {"dialogEvent": {"id": "de:885bec8f-03d7-49b2-bed0-75bbf1d257c2", "speakerUri": "tag:userproxy.com,2025:abc123", "span": {"startTime": "2025-05-10T00:05:06.575947"}, "feat

## UtteranceEvent Specialization

The UtteranceEvent class specializes the Event class. The TextFeature class specializes the Feature class.

In [12]:
from openfloor import UtteranceEvent, To

utterance=DialogEvent(speakerUri="tag:userproxy.com,2025:abc123",features={"text":TextFeature(values=["Hello, world!"])})
utt_event=UtteranceEvent(to=To(speakerUri="tag:userproxy.com,2025:abc123", serviceUrl="https://userproxy.com"),dialogEvent=utterance)

print(utt_event.to_json(indent=2))


{
  "eventType": "utterance",
  "to": {
    "speakerUri": "tag:userproxy.com,2025:abc123",
    "serviceUrl": "https://userproxy.com"
  },
  "parameters": {
    "dialogEvent": {
      "id": "de:327e0d5c-bcbe-481e-8d1b-6648b5f0de7b",
      "speakerUri": "tag:userproxy.com,2025:abc123",
      "span": {
        "startTime": "2025-05-10T00:05:06.640029"
      },
      "features": {
        "text": {
          "mimeType": "text/plain",
          "tokens": [
            {
              "value": "Hello, world!"
            }
          ]
        }
      }
    }
  }
}
